## Some simple algorithms

[author]: # (Written by David Wakeham, May 2020. Please email me at <david.a.wakeham@gmail.com> in case of errors.)

### The Hadamard transform

Morally, quantum computing is powerful because of the parallelism afforded by superposition. Therefore, a common first-step in many quantum algorithms is to prepare a uniform superposition of states. Let's see how to do this!

> ***Exercise 1.*** (a) Suppose we have $n$ qubits in the $|1\rangle$ state, i.e. the state $|1\rangle^{\otimes n}$. Show that applying a Hadamard to each produces
>
> $$H^{\otimes n} |0\rangle^{\otimes n} = \left(\frac{|0\rangle + |1\rangle}{\sqrt{2}}\right)^{\otimes n} = \frac{1}{\sqrt{2^n}}\sum_{x\in \{0,1\}^n}|x\rangle.$$
>
> (b) Explain why this is precisely an even superposition,
>
> $$\left(\frac{|0\rangle + |1\rangle}{\sqrt{2}}\right)^{\otimes n} = \frac{1}{\sqrt{2^n}}\sum_{x\in \{0,1\}^n}|x\rangle.$$
>
> *Hint.* You can proceed  by induction, or instead, take the inner product with $\langle x'|$ for $x' \in \{0,1\}^n$.

We can modify this easily to prepare more interesting superpositions.

> ***Exercise 2.*** (a) Show that for a single qubit, with $x \in \{0, 1\}$, we can write the action of a Hadamard as
>
> $$ H|x\rangle = \frac{|0\rangle + (-1)^x|1\rangle}{\sqrt{2}}.$$
>
> (b) For two qubits in the computational basis, $|x_1\rangle$ and $|x_2\rangle$, $x_1, x_2 \in \{0, 1\}$, verify that
>
> $$ H^{\otimes 2}|x_1\rangle|x_2\rangle = \frac{|00\rangle + (-1)^{x_2}|01\rangle + (-1)^{x_1}|10\rangle + (-1)^{x_1 + x_2}|11\rangle}{\sqrt{2^2}} = \frac{1}{\sqrt{2^2}}\sum_{y \in \{0, 1\}^2} (-1)^{x_1 y_1 + x_2y_2}|y\rangle. $$
>
> (c) Let $x\cdot y = x_1 y_2 + \cdots + x_n y_n$ denote the bitwise inner product for $x, y \in \{0, 1\}^n$. Extend your result from (b) to give
>
> $$H^{\otimes 2}|x\rangle = \frac{1}{\sqrt{2^n}}\sum_{y\in\{0,1\}^n} (-1)^{x\cdot y}|y\rangle.$$
>
> *Hint.* Again, you can either use induction (with base case (a) or (b)) or directly compute the inner product with $\langle y'|$.
>
> (d) Applying $H^{\otimes n}$ is called the *Hadamard transform*. Consider the superposition
>
> $$ |\psi\rangle = \frac{1}{\sqrt{2^n}}\sum_{y\in\{0,1\}^n} (-1)^{f(y)}|y\rangle$$
>
> where $f$ is any binary function $f: \{0, 1\}^n \to \{0, 1\}$. Show that the Hadamard transform of this state is
>
> $$ H^{\otimes n}|\psi\rangle = \frac{1}{\sqrt{2^n}}\sum_{y, z\in\{0,1\}^n} (-1)^{f(y)+y\cdot z}|z\rangle, $$
>
> and deduce from the special case $f(y) = y\cdot x$ that the Hadamard transform is self-inverse.

Since $H^2 = I$, it follows immediately that $H^{\otimes n}$ is self-inverse, so part (d) acts as a check on our superposition formula. The Hadamard transform is closely related to the *Quantum Fourier Transform (QFT)*, one of the central tools in quantum algorithms, to be discussed later in the course.

### Oracles

In a sense, it is "obvious" that quantum parallelism makes quantum computers more powerful than classical computers.
Using a Hadamard transform, I can prepare a superposition of $2^n$ states, and by applying a single $n$-bit unitary $U$, perform exponentially many classical computations in one fell swoop! But linearity giveth and linearity taketh away. When we measure at the end of the day, we cannot learn $U|x\rangle$ for each $x \in \{0, 1\}^n$. Instead, we just get a single number, and it is random. The subtlety and art of quantum computing lies in learning how to use the exponential power of quantum parallelism without throwing everything away when we make our final measurement.

To begin with, let's see how to use parallelism in the "obvious" way mentioned above. Suppose we have a binary function on $n$-bit strings, $f: \{0, 1\}^n \to \{0, 1\}$. We can define a gate $U_f$ which acts on $n+1$ bits. The first $n$ bits store the argument, and the last bit stores the answer:

$$U_f |x, y\rangle = |x, y \oplus f(x)\rangle.$$

This is unitary, and in fact $U_f^2 = I$. Of course, $U_f$ itself might be very hard to construct, but for our purposes, we will simply regard it as a black box of *oracle*. Using $U_f$ is called an *oracle query*. Since the input qubits stores the query, we call it the *query register*.

> ***Exercise 3.*** (a) Show that
>
> $$U_f |x\rangle \left(\frac{|0\rangle - |1\rangle}{\sqrt{2}}\right) = (-1)^{f(x)}|x\rangle\left(\frac{|0\rangle - |1\rangle}{\sqrt{2}}\right).$$
> 
> (b) Prepare the state $|0\rangle^{\otimes n}|1\rangle$. The state $|\psi_f\rangle$ is the result of applying the Hadamard transform and then making an oracle query $U_f$. Confirm that $|\psi_f\rangle$ takes the form
>
> $$ |\psi_f\rangle = U_f H^{\otimes(n+1)}(|0\rangle^{\otimes n}|1\rangle) = \frac{1}{\sqrt{2^n}}\sum_{x\in\{0,1\}^n}(-1)^{f(x)}|x\rangle\left(\frac{|0\rangle-|1\rangle}{\sqrt{2}}\right).$$

Thus, we can perform an exponential number of classical calculations $f(x)$ by applying the Hadamard transform and making a single oracle query. The problem, now, is how to extract anything useful from this, since the information is stored in phases. If we measure the input $n$ qubit, we will learn only a single phase $f(x)$. This is not particularly impressive.

However, if we place some *global* constraints on $f$, involving all the values $f(x)$, it is plausible that by some clever manipulations we can extract global information about $|\psi_f\rangle$ from a single measurement. We will give three algorithms for this below. These constitute the first solid evidence that quantum computers can outperform classical computers.

### Deutsch-Jozsa algorithm

You have already seen Deutsch's algorithm in lectures. The *Deutch-Jozsa algorithm* is a simple extension to the $n$-bit case we are considering. First, we will make the global constraint (or *promise*) that the function $f$ is either *constant* or *balanced*: the function is always the same, or $1$ half the time and $0$ the rest. The algorithm is simply to Hadamard transform and then measure the query register.

> ***Exercise 4.*** (a) Apply the earlier result 2(d) to conclude that
>
> $$H^{\otimes n}|\psi_f\rangle = \frac{1}{2^n}\sum_{x,z\in\{0,1\}^n}(-1)^{f(x)+x\cdot z}|z\rangle\left(\frac{|0\rangle-|1\rangle}{\sqrt{2}}\right).$$
>
> (b) Show that if we measure the query register, the state $|0\rangle^{\otimes n}$ is returned with probability $|\mathcal{A}_0|^2$, where
>
> $$ \mathcal{A}_0 = \langle 0|^{\otimes n}\frac{1}{2^n}\sum_{x,z\in\{0,1\}^n}(-1)^{f(x)+x\cdot z}|z\rangle = \frac{1}{2^n}\sum_{x\in \{0,1\}^n} (-1)^{f(x)}.$$
>
> (c) Argue that if $f$ is constant, then $\mathcal{A}_0 = 1$ and you are guaranteed to return $|0\rangle^{\otimes n}$, while if it is balanced, we have $\mathcal{A}_0 = 0$ and hence we cannot return $|0\rangle^{\otimes n}$.

In other words, when we look at the query register, we can tell whether $f$ is constant or balanced depending on whether we observe $|0\rangle^{\otimes n}$ or not. This algorithm is *exact*, and uses only a single evaluation of $f$. In contrast, the best deterministic classical algorithm requires us to check at least half of the arguments, so $O(2^{n/2})$ function queries! It looks like we have an exponential speedup.

Unfortunately, this is a bit misleading. A slightly fairer comparison is to the *randomized* classical algorithm. Like Deutsch-Jozsa, it takes a constant number $k$ of function queries, with probability of success increasing exponentially with $k$. Deutsch-Jozsa is still faster, but it is really only a "constant" speedup.

### Bernstein–Vazirani algorithm

We can keep playing this game, coming up with a global promise about $f$, and make a clever measurement to determine some associated global information. The *Bernstein–Vazirani algorithm*, for instance, promises that $f$ is a "secret dot product", $f(x) = x\cdot s$ for some secret $s \in \{0, 1\}^n$. As with Deutsch-Jozsa, we Hadamard transform and measure the query register.

> ***Exercise 5.*** Use Exercise 2(d) to conclude that the string $s$ is returned with probability $|\mathcal{A}_s|^2 = 1$.

In other words, the query register contains our answer exactly after a single query. The best classical algorithm requires $n$ queries, on the "basis" strings $b_i = 0^i 1 0^{n-i-2}$, with

$$f(b_i) = s \cdot b_i = s_i.$$

So we reconstruct the "secret" $s$ from the $n$ function calls $f(b_i)$. This is a linear improvement on the best classical algorithm, which is beginning to look like progress. But it's still not exponential.

### Simon's algorithm (bonus)

Based on the preceding examples, it's easy to generalise to a function $f: \{0,1\}^n\to \{0,1\}^n$, and an oracle acting on $x, y \in \{0,1\}^n$ as

$$
U_f |x, y\rangle = |x, y \oplus f(x)\rangle,
$$

where $y \oplus f(x)$ is bitwise as usual. We will prepare a state $|\Psi_f\rangle$, which is slightly different from the above.

> ***Exercise 6.*** Prepare the all-zero state $|0\rangle^{\otimes n}|0\rangle^{\otimes n}$. The parallelized state $|\Psi_f\rangle$ is, as before, prepared by Hadamard transforming the query register and making an oracle query. Show that
>
> $$|\Psi_f\rangle = U_f (H^{\otimes n} \otimes I_n) |0\rangle^{\otimes n}|0\rangle^{\otimes n} = \frac{1}{\sqrt{2^n}}\sum_{x\in\{0,1\}^n} |x\rangle |f(x)\rangle.$$

Once again, to do something useful, we need to make a global promise about $f$ so a single measurement gives nontrivial global information. *Simon's algorithm* starts with the promise that there is a secret string $s \in \{0,1\}^n$ such that, for $x\in \{0,1\}^n$,

$$
f(x) = f(y) \quad \Longleftrightarrow \quad y = x \oplus s.
$$

Like the previous algorithms, our last step is to Hadamard transform the query register, but then we measure the whole circuit.

> ***Exercise 7.*** (a) Let's measure the output register, and suppose the outcome is $|w\rangle$. Explain why the query register is in the state
>
> $$\frac{|x\rangle + |x\oplus s\rangle}{\sqrt{2}}.$$
>
> for some $x \in \{0,1\}^n$.
>
> (b) Now Hadamard transform the query register. Show that it is in the state
>
> $$\frac{1}{\sqrt{2^n}}\sum_{z\in \{0,1\}^n}\left(\frac{(-1)^{x\cdot z}+(-1)^{(x+s)\cdot z}}{\sqrt{2}}\right)|z\rangle.$$
> 
> (c) Finally, measure the output register. Argue that the only terms $|z\rangle$ with nonzero amplitude $\mathcal{A}_z$ satisfy $z \cdot s = 0$.

We have learnt something nontrivial about $s$, namely that it is orthogonal to $z$. If we repeat the circuit multiple times, we will gradually winnow down which subspace it is in, until we can uniquely determine $s$. The number of queries needed is, on average, $O(n)$, since each independent $z$ cuts the space in half, so we go from the whole space of $2^n$ points to $2^{1}$ after we acquired $n-1$ independent $z$. What is the best classical algorithm?

> ***Exercise 8.*** Consider $f: \{0,1\}^n \to \{0,1\}^n$ with the Simon promise, $f(x) = f(y)$ iff $y = x\oplus s$ for a "secret" $s$. Since there is no other structure at hand, the best approach is to randomly guess $x$ and $y$ until we find a "collision", $f(x) = f(y)$. This is called the "birthday problem". Explain why this algorithm takes $O(\sqrt{2^n})$ random guesses.

The best classical (randomized) algorithm is exponential, while Simon's algorithm is $O(n)$. Thus, we have our first example of an exponential quantum speedup! Of course, the problem is rather contrived and useless, which led to reviewers initially rejecting Simon's paper. Scott Aaronson explains what happened next:

> So the story goes that Simon wrote a paper about this theoretical black-box problem with an exponential quantum speedup, and the paper got rejected. But there was one guy who was like, “Hey, this is interesting.” He figured that if you changed a few aspects of what Simon was doing, you could get a quantum algorithm to find the periods of periodic functions, which would in turn let you do all sorts of fun stuff. That guy was Peter Shor.

*Shor's algorithm*, the most famous quantum algorithm, was inspired by Simon's algorithm, and it turns out both are instance of the more general *hidden subgroup problem*. But to understand these generalizations, we first need to understand the generalization of the Hadamard transform: the quantum Fourier transform!